In [48]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

from sklearn.tree import DecisionTreeClassifier

In [2]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
wine.columns

Index(['alcohol', 'sugar', 'pH', 'class'], dtype='object')

In [5]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [8]:
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [9]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [10]:
sub_input.shape, val_input.shape

((4157, 3), (1040, 3))

In [12]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [14]:
scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00797892, 0.00796795, 0.00997162, 0.00997257, 0.00897455]),
 'score_time': array([0.00199509, 0.00099802, 0.00199699, 0.        , 0.00099802]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [15]:
np.mean(scores['test_score'])

0.855300214703487

In [17]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

In [22]:
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [23]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [24]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [25]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [26]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [29]:
gs.cv_results_

{'mean_fit_time': array([0.00917535, 0.02612958, 0.00678139, 0.00678182, 0.00678163]),
 'std_fit_time': array([0.00420385, 0.01460886, 0.00074656, 0.00132292, 0.00116292]),
 'mean_score_time': array([0.00099726, 0.0071815 , 0.00099759, 0.00059881, 0.00099745]),
 'std_score_time': array([6.30374204e-04, 1.23673201e-02, 4.86280395e-07, 4.88928198e-04,
        4.42200589e-07]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.8

In [30]:
np.mean(gs.cv_results_['mean_test_score'])

0.8666152735618569

In [31]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
best_index

0

In [32]:
gs.cv_results_['params'][best_index]

{'min_impurity_decrease': 0.0001}

In [34]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
            'max_depth' : range(5, 20, 1),
            'min_samples_split' : range(2, 100, 10)
}

In [35]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [39]:
gs.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [40]:
gs.best_score_

0.8683865773302731

In [43]:
rgen = randint(0, 10)
rgen.rvs(10)

array([8, 7, 1, 1, 4, 2, 5, 4, 7, 1])

In [44]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([108,  98, 113,  81,  99,  94,  92, 127, 100,  88], dtype=int64))

In [45]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.82034342, 0.53648558, 0.01990747, 0.41611255, 0.95218711,
       0.61266187, 0.51648191, 0.74657381, 0.50036373, 0.13474057])

In [47]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25)
         }

In [49]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D2EA9F8670>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D2EA9425E0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D2EA933850>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001D2EA942910>},
                   random_state=42)

In [50]:
gs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [51]:
gs.best_score_

0.8695428296438884

In [52]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.86